# FSharp Samples

## Downloading Nuget Packages

In [1]:
#!fsharp
#r "nuget: XPlot.Plotly"
#r "nuget: FSharp.Data"
#r "nuget: MathNet.Numerics"

Installed package XPlot.Plotly version 3.0.1

Installed package FSharp.Data version 3.3.3

Installed package MathNet.Numerics version 4.12.0

## Reading a CSV File

In [1]:
#!fsharp
open FSharp.Data
open System.IO

let sp500 = CsvFile.Load(Path.Join(__SOURCE_DIRECTORY__, "./Data/SP500.csv" ))

// Print the prices in the HLOC format
sp500.Rows
|> Seq.take 5

index,Columns
0,"[ 2008-05-02, 1413.900024, 0.0032303750170896706 ]"
1,"[ 2008-05-05, 1407.48999, -0.0045438914840980615 ]"
2,"[ 2008-05-06, 1418.26001, 0.007622806087752565 ]"
3,"[ 2008-05-07, 1392.569946, -0.018279853115200417 ]"
4,"[ 2008-05-08, 1397.680054, 0.0036628357410659618 ]"


## Plotting a Pie Chart

In [1]:
#!fsharp
open XPlot.Plotly

[ "Pizza remaining", 3; "Pizza eaten", 5 ]
|> Chart.Pie

## Plotting a Line Plot

In [1]:
#!fsharp
open XPlot.Plotly
// Source: https://gist.github.com/theprash/7b6163d55b199359cd0dc68c477af0fa 

#r "System.Net.Http"
#r "nuget: Newtonsoft.Json"

let client = new System.Net.Http.HttpClient()
let weatherJson =
    client
        .GetStringAsync("http://www.7timer.info/bin/astro.php?lon=113.2&lat=23.1&ac=0&unit=metric&output=json&tzshift=0")
        .Result

let jObj = Newtonsoft.Json.Linq.JObject.Parse weatherJson

jObj.["dataseries"]
|> Seq.map (fun x -> x.Value<float> "timepoint", x.Value<float> "cloudcover" )
|> Seq.toArray
|> Chart.Line
|> Chart.WithXTitle "Time"
|> Chart.WithYTitle "Cloud cover"

## Plotting 2 Line Plots on One Chart

In [1]:
#!fsharp
open XPlot.Plotly
// Source: https://fslab.org/XPlot/chart/plotly-line-scatter-plots.html

let trace1 =
    Scatter(
        x = [1; 2; 3; 4],
        y = [10; 15; 13; 17]
    )

let trace2 =
    Scatter(
        x = [2; 3; 4; 5],
        y = [16; 5; 11; 9]
    )

[trace1; trace2]
|> Chart.Plot
|> Chart.WithWidth 700
|> Chart.WithHeight 500

## Plotting a Histogram

In [1]:
#!fsharp
open XPlot.Plotly
open MathNet.Numerics.Distributions

let x = 
    ContinuousUniform().Samples()
    |> Seq.take 10000
    |> Seq.toArray

Histogram(x = x) 
|> Chart.Plot
|> Chart.WithWidth 700
|> Chart.WithHeight 500

## The Metropolis-Hastings Algorithm: Iterative with a Fixed Proposal Distribution

In [1]:
#!fsharp
open System
open MathNet.Numerics.Distributions

let gaussianMetropolisHastingsFixedProposal (numberOfIterations: int) 
                                            (mu : double) 
                                            (sigma : double) : double list =
    let mutable states : double list = [] 
    let burnin = int (double numberOfIterations * 0.2)

    let proposedUniform = ContinuousUniform(mu - 5.0 * sigma, mu + 5.0 * sigma)
    let zeroOneUniform  = ContinuousUniform()
    let normal          = Normal(mu, sigma)

    // Initialize the current state.
    let mutable current = proposedUniform.Sample()

    for i in 0..numberOfIterations do

        // Proposed Move
        let proposedMove = proposedUniform.Sample()

        // Acceptance Ratio
        let currentProbability = normal.Density(current)
        let proposedProbabilty = normal.Density(proposedMove) 
        let acceptanceRatio    = Math.Min(proposedProbabilty / currentProbability, 1.0)

        // Choice
        let uniformDraw = zeroOneUniform.Sample() 

        if (uniformDraw < acceptanceRatio) then current <- proposedMove 

        // Append the current state 
        states <- states @ [current]

    states
    |> List.skip(burnin)

let mhRun = gaussianMetropolisHastings 50000 0.0 1.0

Histogram(x = mhRun)
|> Chart.Plot
|> Chart.WithWidth 700
|> Chart.WithHeight 500

In [1]:
#!fsharp
